# Processor

In [ ]:
import torch
from torchvision.models import MobileNetV2
import time

num_class = 10
weight_path = ''
image_size = (224, 224)

In [ ]:
model = MobileNetV2(num_class)

t1 = time.time()
model.load_state_dict(torch.load(weight_path, map_location='cpu')) # Load weight
t2 = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Chọn device nếu là GPU thì sẽ chuyển sang GPU
model.to(device)
model.eval()

print(f'Load weight: {t2 - t1}s')

## 1. Preprocess

In [ ]:
import torch 

def preprocess(card_infos):
    images = [card_info.image for card_info in card_infos]
    samples = [cv2.resize(image, image_size) for image in images]
    samples = np.array(samples)
    samples = torch.from_numpy(samples).to(device).to(torch.float)
    samples = samples.permute(0, 3, 1, 2)
    samples = (samples - samples.mean(dim=(1, 2, 3), keepdim=True)) / samples.std(dim=(1, 2, 3), keepdim=True)
    return card_infos, samples

## 2. Process

In [ ]:
def process(card_infos, samples):
    with torch.no_grad():
        preds = model(samples).softmax(dim=1) # sum preds (dim=1) = 1, probability of each class
    return card_infos, preds

## 3. Postprocess

In [ ]:
bin_threshold = 0.60

In [ ]:
def postprocess(card_infos, preds):
    card_types = [classes[pred.argmax().item()] for pred in preds] # Get idx has highest value 
    scores = [pred[pred.argmax()].item() for pred in preds] # Get prob of class is chosen
    card_types = [card_type if score > bin_threshold else 'other' for card_type, score in zip(card_types, scores)]
    return card_infos, card_types, scores

# Stage

## 1. Prerocess

In [ ]:
def spreprocess(card_infos):
    if __debug__:
        for i, card_info in enumerate(card_infos):
            assert type(card_info.image).__name__ == 'ndarray', f'Image #{i} must be an ndarray.'
            assert card_info.image.ndim == 3, f'Image #{i} must be a 3D ndarray.'
            assert card_info.image.shape[-1] == 3, f'Image #{i} must have 3 channels.'
    return card_infos,

## 2. Process

In [ ]:
def sprocess(card_infos):
    card_infos, samples = preprocess(card_infos)
    card_infos, preds = process(card_infos, samples)
    card_infos, card_types, scores = postprocess(card_infos, preds)
    return card_infos, card_types, scores

## 3. Postprocess

In [ ]:
classes = ['CMND_front', 'CMND_back', 'CCCD_front', 'CCCD_back', 'BLX_front', 'BLX_back', 'CMCC_front', 'PASSPORT', 'PASSPORT_OTHER', 'others']

In [ ]:
def spostprocess(card_infos, card_types, scores):
    for card_info, card_type, score in zip(card_infos, card_types, scores):
        card_info.card_type = card_type
    return card_infos, 

# TEST

In [ ]:
class CardInfo():
    def __init__(self, 
                 idx=None, 
                 original_image=None, 
                 image=None, 
                 warped_size=None, 
                 card_type: str=None, 
                 angle: int=None):
        self.idx = idx
        self.original_image = original_image
        self.image = image
        self.warped_size = warped_size
        self.card_type = card_type
        self.angle = angle

In [ ]:
import cv2
import numpy as np

image = cv2.imread('test_images/input/extracted_card.jpg')
card_info = CardInfo()

card_infos = []
card_info.image = image
card_infos.append(card_info)

In [ ]:
card_infos, = spreprocess(card_infos)
card_infos, card_types, scores = sprocess(card_infos)
card_infos, = spostprocess(card_infos, card_types, scores)

In [ ]:
cv2.imshow('image', image)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
print(card_infos[0].card_type)